Intro

Step 1: Import training image dataset

In [3]:
#import kagglehub

#path = kagglehub.dataset_download("feyzazkefe/trashnet")
#print("Path to dataset files:", path)

Step 2: Import packages

In [4]:
import numpy as np
from PIL import Image

from collections.abc import Iterable

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.models import efficientnet_b0
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from collections.abc import Iterable
from sklearn.model_selection import train_test_split

Step 2: Split data into training and test sets & transform both sets to tensors

In [5]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to EfficientNet input size
    transforms.RandomHorizontalFlip(),  # Lightweight and effective
    transforms.RandomRotation(10),  # Augment slightly with small angles
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to EfficientNet input size
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

In [6]:
# Define training data
train_data = torchvision.datasets.ImageFolder(root='./data/train', transform=transform_train)
test_data = torchvision.datasets.ImageFolder(root='./data/test', transform=transform_test)

# Define data loaders & batch size
batch_size = 32 # Can also be 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)


Step 3: Load in our efficietnet_b0 model & define number of classes in final layer

In [7]:
# Load in efficientnet_b0
model = efficientnet_b0(pretrained=True)

# Define class number
num_classes = 6
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Print class names from training data
print("Classes:", train_data.classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


/home/platinumfish/miniconda3/envs/Climate_CNN_3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/platinumfish/miniconda3/envs/Climate_CNN_3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Step 4: Define loss function & optimizer, as well as training loop and model evaluation method

In [8]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize weights

            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(dataloader):.4f}")

# Model evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
train_model(model, train_loader, criterion, optimizer, epochs=10)
evaluate_model(model, test_loader)